In [ ]:
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

ref = dt.date(2024,6,30)

file_path = '/content/'
file_prod_CM = file_path + 'Prod_06_2024.xlsx'
file_prod_PM = file_path + 'Produção - Maio.xlsx'

file_bodereaux_CM = file_path + 'bordereaux_cessiones20240703000304.csv'
file_portfolio_CM = file_path + 'solvency_portfolio20240702235637.csv'

file_DW_CM = file_path + 'DW_Jun_06_Apol_Frac.csv'

file_tabua_2023 = file_path + 'D_tabua_a_partir_2023.xlsx'
file_tabua_lt_2022 = file_path + 'D_tabua_anterior_dez_2022.xlsx'
file_tabua_others = file_path + 'D_tabua_others.xlsx'


df_prod_CM = pd.read_excel(file_prod_CM)
df_prod_PM = pd.read_excel(file_prod_PM)

df_bodereaux_CM = pd.read_csv(file_bodereaux_CM, delimiter=';')
df_portfolio_CM = pd.read_csv(file_portfolio_CM, delimiter=';')

df_DW_CM = pd.read_csv(file_DW_CM, delimiter=';')

df_tabua_2023 = pd.read_excel(file_tabua_2023)
df_tabua_lt_2022 = pd.read_excel(file_tabua_lt_2022)
df_tabua_others = pd.read_excel(file_tabua_others)


In [ ]:
df_portfolio_CM_0 = df_portfolio_CM.copy()

df_portfolio_CM_0['Chave2'] = df_portfolio_CM_0['num_apolice'].astype('str') + df_portfolio_CM_0['dt_nascimento'].astype('str') + df_portfolio_CM_0['sexo']

df_portfolio_CM_0['produto'] = df_portfolio_CM_0['id_produto'].astype('int')

df_portfolio_CM_0 = df_portfolio_CM_0[['Chave2', 'produto', 'perc_agrav_morte', 'perm_agrav_morte']]

df_portfolio_CM_1 = df_portfolio_CM_0.drop_duplicates()

df_bodereaux_CM_0 = df_bodereaux_CM.copy()

df_bodereaux_CM_0['Chave2'] = df_bodereaux_CM_0['NO_POLIZA'].astype('str') + df_bodereaux_CM_0['FECHA_NACIMIENTO'].astype('str') + df_bodereaux_CM_0['SEXO']

df_bodereaux_CM_0['FECHA_EFECTO_POLIZA'] = pd.to_datetime(df_bodereaux_CM_0['FECHA_EFECTO_POLIZA'])
df_bodereaux_CM_0['FECHA_NACIMIENTO'] = pd.to_datetime(df_bodereaux_CM_0['FECHA_NACIMIENTO'])


In [ ]:
print(f'df_prod_CM shape: {df_prod_CM.shape}')
print(f'df_prod_PM shape: {df_prod_PM.shape}')
print(f'df_bodereaux_CM shape: {df_bodereaux_CM.shape}')
print(f'df_portfolio_CM shape: {df_portfolio_CM.shape}')
print(f'df_DW_CM shape: {df_DW_CM.shape}')

In [ ]:
df_bodereaux_CM_1 = pd.merge(df_bodereaux_CM_0,df_portfolio_CM_1, how='left', on='Chave2')

In [ ]:
# CHECK MISSING NO JOIN DA BASE PORTFOLIO CURRENT MONTH
df_bodereaux_CM_1[df_bodereaux_CM_1['produto'].isna()]

In [ ]:
# INSERIR VALORES NO MISSINGS - CORRER SE TIVER NECESSIDADE, CASO CONTRÁRIO PULE PARA O CODIGO SEGUINTE

df_bodereaux_CM_1['produto'] = df_bodereaux_CM_1['produto'].fillna(100)
df_bodereaux_CM_1['perc_agrav_morte'] = df_bodereaux_CM_1['perc_agrav_morte'].fillna(0)
df_bodereaux_CM_1['perm_agrav_morte'] = df_bodereaux_CM_1['perm_agrav_morte'].fillna(0)

In [ ]:
df_DW_CM_1 = df_DW_CM.drop_duplicates()
df_bodereaux_CM_2 = pd.merge(df_bodereaux_CM_1,df_DW_CM_1, how='left', left_on='NO_POLIZA', right_on='Apol')

In [ ]:
# CHECK MISSING NO JOIN DA BASE PORTFOLIO CURRENT MONTH
df_bodereaux_CM_1[df_bodereaux_CM_1['produto'].isna()]

In [ ]:
# INSERIR VALORES NO MISSINGS - CORRER SE TIVER NECESSIDADE, CASO CONTRÁRIO PULE PARA O CODIGO SEGUINTE
df_bodereaux_CM_2['Fracionamento'] = df_bodereaux_CM_2['Fracionamento'].fillna('M')

In [ ]:
df_bodereaux_CM_2.drop('Apol', axis='columns', inplace=True)

In [ ]:
df_bodereaux_CM_2['EDAD_PLUS_1'] = df_bodereaux_CM_2['EDAD'] + 1
df_bodereaux_CM_2['EDAD_MINUS_1'] = df_bodereaux_CM_2['EDAD'] -1

df_tabuas_0 = pd.merge(df_tabua_2023,df_tabua_lt_2022, how='left', on ='IDADE')
df_tabuas = pd.merge(df_tabuas_0,df_tabua_others, how='left', on ='IDADE')

df_bodereaux_CM_3 = pd.merge(df_bodereaux_CM_2,df_tabuas, how='left', left_on='EDAD_PLUS_1', right_on='IDADE')
df_bodereaux_CM_4 = pd.merge(df_bodereaux_CM_3,df_tabuas, how='left', left_on='EDAD_MINUS_1', right_on='IDADE')
df_bodereaux_CM_5 = pd.merge(df_bodereaux_CM_4,df_tabuas, how='left', left_on='EDAD', right_on='IDADE')

df_bodereaux_CM_5.info()

In [ ]:
df_bodereaux_CM_5.shape

In [ ]:
# 'IDADE_x' = IDADE + 1 / EDAD_PLUS_1
# 'IDADE_y' = IDADE - 1 / EDAD_MINUS_1
# 'IDADE' = IDADE / EDAD

df_bodereaux_CM_5.drop(['IDADE_x', 'IDADE_y', 'IDADE'], axis='columns', inplace=True)

In [ ]:
# VERIFICAR O NOME DO CAMPO E O TIPO DA VARIAVEL: Nª APOLICE E DATA RENOVAÇÃO SEGURO
df_prod_CM.info()

In [ ]:
# VERIFICAR O NOME DO CAMPO E O TIPO DA VARIAVEL: Nª APOLICE E DATA RENOVAÇÃO SEGURO
df_prod_PM.info()

In [ ]:
# ALTERAR O NOME DO CAMPO Nº DA APOLICE SE FOR NECESSARIO

df_prod_CM_0 = df_prod_CM[['Nºdaapólice', 'Data Renovação Seguro']].copy()
df_prod_PM_0 = df_prod_PM[['Nº da apólice', 'Data Renovação Seguro']].copy()

In [ ]:
# ALTERAR O NOME DO CAMPO Nº DA APOLICE SE FOR NECESSARIO

df_prod_PM_0['Nº da apólice'] = df_prod_PM_0['Nº da apólice'].apply(lambda x: x.replace(" ", "")).astype('int')

In [ ]:
# ALTERAR O NOME DO CAMPO Nº DA APOLICE SE FOR NECESSARIO

df_prod_CM_1 = df_prod_CM_0.drop_duplicates()
df_prod_CM_1.rename(columns={'Nºdaapólice': 'Apol', 'Data Renovação Seguro': 'Dt_Renov_CM'}, inplace=True)


df_prod_PM_1 = df_prod_PM_0.drop_duplicates()
df_prod_PM_1.rename(columns={'Nº da apólice': 'Apol', 'Data Renovação Seguro': 'Dt_Renov_PM'}, inplace=True)

df_prod_PM_1.info()

In [ ]:
df_bodereaux_CM_6 = pd.merge(df_bodereaux_CM_5,df_prod_PM_1, how='left', left_on='NO_POLIZA', right_on='Apol')
df_bodereaux_CM_7 = pd.merge(df_bodereaux_CM_6,df_prod_CM_1, how='left', left_on='NO_POLIZA', right_on='Apol')
df_bodereaux_CM_7.drop(['Apol_x', 'Apol_y'], axis='columns', inplace=True)
df_bodereaux_CM_7.info()

In [ ]:
# CHECK MISSING NO JOIN DA BASE PRODUÇÃO

df_bodereaux_CM_7[(df_bodereaux_CM_7['Dt_Renov_PM'].isna()) & (df_bodereaux_CM_7['Dt_Renov_CM'].isna())]['PERIODO'].count()

In [ ]:
df_bodereaux_CM_7[(df_bodereaux_CM_7['Dt_Renov_PM'].isna()) & (df_bodereaux_CM_7['Dt_Renov_CM'].isna())].head()

In [ ]:
df_bodereaux_CM_8 = df_bodereaux_CM_7.copy()

for lab, row in df_bodereaux_CM_8.iterrows():
    # Condition 1: If both Dt_Renov_CM and Dt_Renov_PM are NaN
    if pd.isna(df_bodereaux_CM_8.loc[lab, 'Dt_Renov_CM']) and pd.isna(df_bodereaux_CM_8.loc[lab, 'Dt_Renov_PM']):
        df_bodereaux_CM_8.loc[lab, 'Dt_Renov'] = ref
    # Condition 2: If Dt_Renov_CM is NaN but Dt_Renov_PM is available
    elif pd.isna(df_bodereaux_CM_8.loc[lab, 'Dt_Renov_PM']):
        # Extract month and day directly from the string
        month = int(df_bodereaux_CM_8.loc[lab, 'Dt_Renov_CM'][:2])  # First 2 characters as month
        day = int(df_bodereaux_CM_8.loc[lab, 'Dt_Renov_CM'][-2:])   # Last 2 characters as day
        df_bodereaux_CM_8.loc[lab, 'Dt_Renov'] = dt.date(ref.year, month, day)
    # Condition 3: If Dt_Renov_PM is NaN but Dt_Renov_CM is available
    elif pd.isna(df_bodereaux_CM_8.loc[lab, 'Dt_Renov_CM']):
        # Extract month and day directly from the string
        month = int(df_bodereaux_CM_8.loc[lab, 'Dt_Renov_PM'][:2])  # First 2 characters as month
        day = int(df_bodereaux_CM_8.loc[lab, 'Dt_Renov_PM'][-2:])   # Last 2 characters as day
        df_bodereaux_CM_8.loc[lab, 'Dt_Renov'] = dt.date(ref.year, month, day)


In [ ]:
# VALIDAÇÃO DOS CASOS QUE NÃO ESTÃO NA PRODUÇÃO DO MES ATUAL E ANTERIOR

df_bodereaux_CM_8[(df_bodereaux_CM_8['Dt_Renov_PM'].isna() == True) & (df_bodereaux_CM_8['Dt_Renov_CM'].isna() == True)].head()

In [ ]:
df_validation_0 = df_bodereaux_CM_8.copy()
df_validation_0.drop(['Dt_Renov_PM', 'Dt_Renov_CM'], axis='columns', inplace=True)

In [ ]:
# CRIACAO DOS CHECKS CAPITAL
df_validation_0 = df_bodereaux_CM_8.copy()
df_validation_0.drop(['Dt_Renov_PM', 'Dt_Renov_CM'], axis='columns', inplace=True)

df_validation_0['CHECK_CAP_MORTE'] = round(df_validation_0['CAPITAL_SEGURO_COB_MORTE'] - df_validation_0['CAPITAL_RETIDO_COB_MORTE'] - df_validation_0['CAPITAL_EXCEDENTE_COB_MORTE'] - df_validation_0['CAPITAL_FAC_COB_MORTE'],6)
df_validation_0['FLAG_CAP_MORTE'] = df_validation_0['CHECK_CAP_MORTE'].apply(lambda x: 'N' if abs(round(x,1)) > 0 else 'Y')

df_validation_0['CHECK_CAP_MAC'] = round(df_validation_0['CAPITAL_SEGURO_COB_MAC'] - df_validation_0['CAPITAL_RETIDO_COB_MAC'] - df_validation_0['CAPITAL_EXCEDENTE_COB_MAC'] - df_validation_0['CAPITAL_FAC_COB_MAC'],6)
df_validation_0['FLAG_CAP_MAC'] = df_validation_0['CHECK_CAP_MAC'].apply(lambda x: 'N' if abs(round(x,1)) > 0 else 'Y')

df_validation_0['CHECK_CAP_IAD'] = round(df_validation_0['CAPITAL_SEGURO_COB_IAD'] - df_validation_0['CAPITAL_RETIDO_COB_IAD'] - df_validation_0['CAPITAL_EXCEDENTE_COB_IAD'] - df_validation_0['CAPITAL_FAC_COB_IAD'],6)
df_validation_0['FLAG_CAP_IAD'] = df_validation_0['CHECK_CAP_IAD'].apply(lambda x: 'N' if abs(round(x,1)) > 0 else 'Y')

df_validation_0['CHECK_CAP_ITP'] = round(df_validation_0['CAPITAL_SEGURO_COB_ITP'] - df_validation_0['CAPITAL_RETIDO_COB_ITP'] - df_validation_0['CAPITAL_EXCEDENTE_COB_ITP'] - df_validation_0['CAPITAL_FAC_COB_ITP'],6)
df_validation_0['FLAG_CAP_ITP'] = df_validation_0['CHECK_CAP_ITP'].apply(lambda x: 'N' if abs(round(x,1)) > 0 else 'Y')

df_validation_0['CHECK_CAP_DG'] = round(df_validation_0['CAPITAL_SEGURO_COB_DG'] - df_validation_0['CAPITAL_RETIDO_COB_DG'] - df_validation_0['CAPITAL_EXCEDENTE_COB_DG'] - df_validation_0['CAPITAL_FAC_COB_DG'],6)
df_validation_0['FLAG_CAP_DG'] = df_validation_0['CHECK_CAP_DG'].apply(lambda x: 'N' if abs(round(x,1)) > 0 else 'Y')


In [ ]:
df_validation_0[df_validation_0['FLAG_CAP_MAC'] == 'N']

In [ ]:
# CRIACAO DAS TARIFAS

# 2023 = tabua 2023 P:X
# 1   FALLECIMIENTO_60% PASEM 2010_M  57 non-null     float64
# 2   FALLECIMIENTO_60% PASEF 2010_F  57 non-null     float64
# 3   IAD_95% Tabla BPI Vida_M        52 non-null     float64
# 4   IAD_95% Tabla BPI Vida_F        52 non-null     float64
# 5   ITP_60% TNI_M                   47 non-null     float64
# 6   ITP_60% TNI_F                   47 non-null     float64
# 7   EEGG_TABLA Gen Re_M             47 non-null     float64
# 8   EEGG_TABLA Gen Re_F             47 non-null     float64

# 2022 = tabua 2022 A:J
# 9   MORTE_65% PASEMM                57 non-null     float64
# 10  MORTE_65% PASEMF                57 non-null     float64
# 11  IAD_TABUA PROPRIA_M             52 non-null     float64
# 12  IAD_TABUA PROPRIA_F             52 non-null     float64
# 13  ITP_75% GKM80                   47 non-null     float64
# 14  ACIDENTES_0,5 MIL               52 non-null     float64
# 15  ACIDENTES CIRCULACAO_0,23 MIL   52 non-null     float64
# 16  DOENCAS GRAVES_TABUA PROPRIA_M  47 non-null     float64
# 17  DOENCAS GRAVES_TABUA PROPRIA_F  47 non-null     float64

# OTHERS = OTHERS AM:AR
# 18  FALLECIMIENTO_M                 57 non-null     float64
# 19  FALLECIMIENTO_F                 57 non-null     float64
# 20  IAD_M                           52 non-null     float64
# 21  IAD_F                           52 non-null     float64
# 22  Unsex                           47 non-null     float64

#if ((df_validation_0.loc[lab, 'produto'] == 100) | (df_validation_0.loc[lab, 'produto'] == 104)) & (int(df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA'][:4]) < 2023):

# 'IDADE_x' = IDADE + 1 / EDAD_PLUS_1
# 'IDADE_y' = IDADE - 1 / EDAD_MINUS_1
# 'IDADE' = IDADE / EDAD

# OTHERS
#=IF(AND(OR(BH3=100;BH3=104);YEAR(D3)<2023);
#    IF(H3="M";VLOOKUP(G3;'Tarifa Resseguro'!$AM$6:$AO$62;2;0);
#    VLOOKUP(G3;'Tarifa Resseguro'!$AM$6:$AO$62;3;0));
#
# Quadro 1 Contrato Resseguro - Tasas reaseguro para Negocio Vinculado de nueva producción a partir de 1.1.2023
#    IF(AND(YEAR(D3)>=2023;LEFT(C3;1)*1=1);
#       IF(H3="M";VLOOKUP(G3;'Tarifa Resseguro'!$P$6:$X$62;2;FALSE);
#       VLOOKUP(G3;'Tarifa Resseguro'!$P$6:$X$62;3;FALSE));
#
# Quadro 2 Contrato Resseguro - Tasas reaseguro Negocio No Vinculado nueva producción y cartera y de Negocio Vinculado de cartera hasta 31.12.2022:
#       IF(H3="M";VLOOKUP(G3;'Tarifa Resseguro'!$A$6:$J$62;2;FALSE);VLOOKUP(G3;'Tarifa Resseguro'!$A$6:$J$62;3;FALSE))))


for lab, row in df_validation_0.iterrows():

    if ((df_validation_0.loc[lab, 'produto'] == 100) | (df_validation_0.loc[lab, 'produto'] == 104)) & (int(df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA'].strftime('%Y')) < 2023):
      if (df_validation_0.loc[lab, 'SEXO'] == 'M'):
        df_validation_0.loc[lab, 'Tarifa'] = df_validation_0.loc[lab, 'FALLECIMIENTO_M']
      else:
        df_validation_0.loc[lab, 'Tarifa'] = df_validation_0.loc[lab, 'FALLECIMIENTO_F']

    elif (int(df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA'].strftime('%Y')) > 2023) & (str(df_validation_0.loc[lab, 'NO_POLIZA'])[:1] == '1'):
      if (df_validation_0.loc[lab, 'SEXO'] == 'M'):
        df_validation_0.loc[lab, 'Tarifa'] = df_validation_0.loc[lab, 'FALLECIMIENTO_60% PASEM 2010_M']
      else:
        df_validation_0.loc[lab, 'Tarifa'] = df_validation_0.loc[lab, 'FALLECIMIENTO_60% PASEF 2010_F']

    else:
      if (df_validation_0.loc[lab, 'SEXO'] == 'M'):
        df_validation_0.loc[lab, 'Tarifa'] = df_validation_0.loc[lab, 'MORTE_65% PASEMM']
      else:
        df_validation_0.loc[lab, 'Tarifa'] = df_validation_0.loc[lab, 'MORTE_65% PASEMF']

    if ((df_validation_0.loc[lab, 'produto'] == 100) | (df_validation_0.loc[lab, 'produto'] == 104)) & (int(df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA'].strftime('%Y')) < 2023):
      if (df_validation_0.loc[lab, 'SEXO'] == 'M'):
        df_validation_0.loc[lab, 'Tarifa_PLUS_1'] = df_validation_0.loc[lab, 'FALLECIMIENTO_M_x']
      else:
        df_validation_0.loc[lab, 'Tarifa_PLUS_1'] = df_validation_0.loc[lab, 'FALLECIMIENTO_F_x']

    elif (int(df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA'].strftime('%Y')) > 2023) & (str(df_validation_0.loc[lab, 'NO_POLIZA'])[:1] == '1'):
      if (df_validation_0.loc[lab, 'SEXO'] == 'M'):
        df_validation_0.loc[lab, 'Tarifa_PLUS_1'] = df_validation_0.loc[lab, 'FALLECIMIENTO_60% PASEM 2010_M_x']
      else:
        df_validation_0.loc[lab, 'Tarifa_PLUS_1'] = df_validation_0.loc[lab, 'FALLECIMIENTO_60% PASEF 2010_F_x']

    else:
      if (df_validation_0.loc[lab, 'SEXO'] == 'M'):
        df_validation_0.loc[lab, 'Tarifa_PLUS_1'] = df_validation_0.loc[lab, 'MORTE_65% PASEMM_x']
      else:
        df_validation_0.loc[lab, 'Tarifa_PLUS_1'] = df_validation_0.loc[lab, 'MORTE_65% PASEMF_x']


    if df_validation_0.loc[lab, 'EDAD']  > 18:
      if ((df_validation_0.loc[lab, 'produto'] == 100) | (df_validation_0.loc[lab, 'produto'] == 104)) & (int(df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA'].strftime('%Y')) < 2023):
        if (df_validation_0.loc[lab, 'SEXO'] == 'M'):
          df_validation_0.loc[lab, 'Tarifa_MINUS_1'] = df_validation_0.loc[lab, 'FALLECIMIENTO_M_y']
        else:
          df_validation_0.loc[lab, 'Tarifa_MINUS_1'] = df_validation_0.loc[lab, 'FALLECIMIENTO_F_y']

      elif (int(df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA'].strftime('%Y')) > 2023) & (str(df_validation_0.loc[lab, 'NO_POLIZA'])[:1] == '1'):
        if (df_validation_0.loc[lab, 'SEXO'] == 'M'):
          df_validation_0.loc[lab, 'Tarifa_MINUS_1'] = df_validation_0.loc[lab, 'FALLECIMIENTO_60% PASEM 2010_M_y']
        else:
          df_validation_0.loc[lab, 'Tarifa_MINUS_1'] = df_validation_0.loc[lab, 'FALLECIMIENTO_60% PASEF 2010_F_y']

      else:
        if (df_validation_0.loc[lab, 'SEXO'] == 'M'):
          df_validation_0.loc[lab, 'Tarifa_MINUS_1'] = df_validation_0.loc[lab, 'MORTE_65% PASEMM_y']
        else:
          df_validation_0.loc[lab, 'Tarifa_MINUS_1'] = df_validation_0.loc[lab, 'MORTE_65% PASEMF_y']

    else:
      df_validation_0.loc[lab, 'Tarifa_MINUS_1'] = 0

In [ ]:
# D3 = FECHA_EFECTO_POLIZA
# $O$1 = 30/06/2024
# BF3 = Agrav %
# BG3 = Agrav %0
# BR3 = dt renovação 2023

# SUM(K3:L3) = CAPITAL_EXCEDENTE_COB_MORTE + CAPITAL_FAC_COB_MORTE
# O3 = Tarifa



# =+IF(
      #AND($O$1-D3<365,25;
          #BR3<=$O$1;
          #MONTH(BR3)=MONTH($O$1);
          #D3<BR3;
          #IF(MONTH($O$1)=12;
                              #MONTH(D3)=1;
                              #MONTH(D3)=MONTH($O$1)+1
            #)
          #);

    #SUM(K3:L3)*O3;

    #IF($O$1-D3<365,25;
        #SUM(K3:L3)*O3*0,5;
        #SUM(K3:L3)*O3))*BF3 +(SUM(K3:L3)/1000*BG3)

  df_validation_1 = df_validation_0.copy()


  for lab, row in df_validation_1.iterrows():
    if (ref.strftime('%M') = 12):

      #if ((df_validation_0.loc[lab, 'produto'] == 100) | (df_validation_0.loc[lab, 'produto'] == 104)) & (int(df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA'].strftime('%Y')) < 2023):
      if ((ref - df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA']) < 365.25) & (df_validation_0.loc[lab, 'Dt_Renov'] <= ref) &
      (df_validation_0.loc[lab, 'Dt_Renov'].strftime('%M') == ref.strftime('%M')) & (df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA'] < df_validation_0.loc[lab, 'Dt_Renov']) &
      (df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA'].strftime('%M') == 1):

        df_validation_0.loc[lab, 'Premio_Morte'] = df_validation_0.loc[lab, 'Tarifa'] * (df_validation_0.loc[lab, 'CAPITAL_EXCEDENTE_COB_MORTE'] + df_validation_0.loc[lab, 'CAPITAL_FAC_COB_MORTE'])

      else:
        if  ((ref - df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA']) < 365.25):
          df_validation_0.loc[lab, 'Premio_Morte'] = df_validation_0.loc[lab, 'Tarifa'] * (df_validation_0.loc[lab, 'CAPITAL_EXCEDENTE_COB_MORTE'] + df_validation_0.loc[lab, 'CAPITAL_FAC_COB_MORTE']) * 0.5
        else:







    else:
      if ((ref - df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA']) < 365.25) & (df_validation_0.loc[lab, 'Dt_Renov'] <= ref) &
      (df_validation_0.loc[lab, 'Dt_Renov'].strftime('%M') == ref.strftime('%M')) & (df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA'] < df_validation_0.loc[lab, 'Dt_Renov']) &
      (df_validation_0.loc[lab, 'FECHA_EFECTO_POLIZA'].strftime('%M') == (ref.strftime('%M') + 1)) :

        df_validation_0.loc[lab, 'Premio_Morte'] = df_validation_0.loc[lab, 'Tarifa'] * (df_validation_0.loc[lab, 'CAPITAL_EXCEDENTE_COB_MORTE'] + df_validation_0.loc[lab, 'CAPITAL_FAC_COB_MORTE'])





        df_validation_0.loc[lab, 'Tarifa'] = df_validation_0.loc[lab, 'FALLECIMIENTO_M']
      else:
        df_validation_0.loc[lab, 'Tarifa'] = df_validation_0.loc[lab, 'FALLECIMIENTO_F']